## Bibliotecas

In [78]:
import os
import pandas as pd
from docx import Document
import openai
from openai import OpenAI
import pandas as pd



In [ ]:

API_KEY = ""
openai.api_key = API_KEY
client = OpenAI(api_key=API_KEY)

## Variáveis

In [ ]:
pastas = ["SPA", "BHV", "BHT", "SPP", "SPU","extras"]


perguntas = [
    "1. Acesso relacionado ao preço (o alimento fresco e saudável é acessível do ponto de vista financeiro?)",
    "2. Acesso relacionado a quantidade de alimento fresco e saudável ofertado na região (a oferta de alimentos frescos e saudáveis é suficiente para a região?)",
    "3. Acesso relacionado a distância (o alimento fresco e saudável é facilmente encontrado em locais próximos?)",
    "4. Acesso relacionado a adaptabilidade das necessidades locais (horários de abertura dos estabelecimentos, meios de pagamento estão alinhados com a necessidade dos moradores locais para compra de alimentos frescos e saudáveis?)",
    "5. Acesso relacionado a aceitabilidade do alimento (o alimento é do gosto e qualidade dos moradores das regiões pesquisadas?)",
    "6. Preferência e consumo (Preferem e consomem alimentos saudáveis ou não saudáveis? Quais os motivos para essa preferência e consumo?)",
    "7. Percepções sobre alimento/dieta saudável (O que é um alimento saudável (vs. não saudável)? O que consideram ser uma dieta saudável (vs. não saudável)?)",
    "8. Quando consomem (Em quais contextos e ocasiões consomem alimentos saudáveis (vs. não saudáveis)?)",
    "9. Influências externas (Quais são as influências culturais e sociais na compra e consumo de alimentos saudáveis (vs. alimentos não saudáveis)?)",
    "10. Emoções e psicológico (Quais são as motivações emocionais e psicológicas para consumo e compra de alimentos saudáveis (vs. alimentos não saudáveis)?)",
]

system_message = """
Você é um assistente que analisa textos e responde perguntas relacionadas à análise de alimentos frescos e saudáveis.
Responda cada pergunta de forma clara e objetiva com base apenas no que foi apresentado nos arquivos.
"""

resultados = []


## Função para ler os arquivos

In [80]:
def ler(caminho):
    try:
        doc = Document(caminho)
        conteudo = "\n".join([p.text for p in doc.paragraphs if p.text.strip()])
        return conteudo
    except Exception as e:
        print(f"Erro ao ler o arquivo {caminho}: {e}")
        return None

## Função da API

In [81]:
def get_completion(prompt, system_message, model="gpt-4-turbo-2024-04-09"):
    client = OpenAI(api_key=API_KEY)
    messages = [
        {'role': 'system', 'content': system_message},
        {"role": "user", "content": prompt},
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.01,
    )
    response_message = response.choices[0].message.content
    return response_message

## Processamento das perguntas

In [66]:
for pasta in pastas:
    if not os.path.exists(pasta):
        print(f"Pasta {pasta} não encontrada, ignorando...")
        continue

    for filename in os.listdir(pasta):
        if filename.endswith(".docx"):
            caminho = os.path.join(pasta, filename)
            
            # Lê o conteúdo do arquivo
            content = ler(caminho)
            if content:
                row = {"Pasta": pasta, "Arquivo": filename}
                for i, question in enumerate(perguntas, start=1):
                    try:
                        # Aqui, você pode chamar a função `get_completion` para obter as respostas
                        response = get_completion(content, system_message + f" Responda: {question}")
                        row[f"Pergunta {i}"] = response
                        print(f"deu certo ao processar o arquivo {filename} na pasta {pasta} para a pergunta {i}, resposta: {response}")
        
                    except Exception as error:
                        print(f"Erro ao processar o arquivo {filename} na pasta {pasta} para a pergunta {i}: {error}")
                        row[f"Pergunta {i}"] = f"Erro ao processar: {error}"

                resultados.append(row)

Erro ao processar o arquivo 20210304 - BHT45 - Leandra (Interview 2).docx na pasta BHT para a pergunta 1: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Erro ao processar o arquivo 20210304 - BHT45 - Leandra (Interview 2).docx na pasta BHT para a pergunta 2: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Erro ao processar o arquivo 20210304 - BHT45 - Leandra (Interview 2).docx na pasta BHT para a pergunta 3: Error code: 429 - {'error': {'message': 'You exceeded your c

KeyboardInterrupt: 

## Planilha

In [ ]:
df = pd.DataFrame(resultados)
df.to_excel("SPU_respostas2.xlsx", index=False)
print(df)

  Pasta                                        Arquivo  \
0   SPU       20210218 - União - GF4(Transcript).docx   
1   SPU  20210223 - SPU16 - Adriele (Interview 2).docx   
2   SPU   20210305 - SPU37 - Marina (Interview 2).docx   
3   SPU   20210306 - SPU40 - Denise (Interview 2).docx   
4   SPU   20210308 - SPU34 - Simone (Interview 2).docx   

                                          Pergunta 1  \
0  O acesso ao alimento fresco e saudável no bair...   
1  O acesso ao alimento fresco e saudável, do pon...   
2  O acesso ao alimento fresco e saudável, do pon...   
3  O acesso ao alimento fresco e saudável parece ...   
4  O acesso ao alimento fresco e saudável, do pon...   

                                          Pergunta 2  \
0  O texto não fornece informações específicas so...   
1  O texto não fornece informações específicas so...   
2  A transcrição não fornece informações suficien...   
3  A transcrição não fornece informações suficien...   
4  A oferta de alimentos frescos e